In [1]:
!python -V

Python 3.11.7


In [2]:
import pandas as pd

In [3]:
pwd()

'/workspaces/mlops-zc/01-intro'

In [32]:
df1 = pd.read_parquet('../notebooks/data/green_tripdata_2021-01.parquet')
df2 = pd.read_parquet('../notebooks/data/green_tripdata_2021-02.parquet')

In [33]:
print(df1.dtypes)

VendorID                          int64
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
store_and_fwd_flag               object
RatecodeID                      float64
PULocationID                      int64
DOLocationID                      int64
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                        object
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
dtype: object


In [ ]:
# print(df1)
# print(df2)

In [36]:
# print(df1.dtypes)
# print(df2.dtypes)
df1 = df1[df1.trip_type == 2]
df2 = df2[df2.trip_type == 2]

In [37]:
# print(df1)
# print(df2)

In [44]:
# unnecessary with parquet files:
# df1.lpep_dropoff_datetime = pd.to_datetime(df1.lpep_dropoff_datetime)
# df1.lpep_pickup_datetime = pd.to_datetime(df1.lpep_pickup_datetime)

In [55]:
df1['duration'] = df1.lpep_dropoff_datetime - df1.lpep_pickup_datetime
df2['duration'] = df2.lpep_dropoff_datetime - df2.lpep_pickup_datetime

In [56]:
print(df1.duration)

6       0 days 00:00:07
15      0 days 00:00:02
30      0 days 00:19:46
43      0 days 02:01:12
45      0 days 00:00:04
              ...      
40363   0 days 00:09:27
40364   0 days 00:35:02
40377   0 days 00:00:40
40394   0 days 00:00:03
40430   0 days 00:23:09
Name: duration, Length: 1238, dtype: timedelta64[us]


In [57]:
df1.duration = df1.duration.apply(lambda td: td.total_seconds() / 60)
df2.duration = df2.duration.apply(lambda td: td.total_seconds() / 60)

In [58]:
df1.duration

6          0.116667
15         0.033333
30        19.766667
43       121.200000
45         0.066667
            ...    
40363      9.450000
40364     35.033333
40377      0.666667
40394      0.050000
40430     23.150000
Name: duration, Length: 1238, dtype: float64

In [63]:
print(df1.duration.min())
print(df2.duration.min())

0.0
0.0
